In [12]:
!pwd
%cd /Users/karinna_cv/SpatialScaper/SpatialScaper

/Users/karinna_cv/SpatialScaper/SpatialScaper
/Users/karinna_cv/SpatialScaper/SpatialScaper


In [14]:
!python scripts/prepare_rirs.py

Overwriting datasets/rir_datasets/spatialscaper_RIRs/motusroom_foa.sofa
SOFA file saved to datasets/rir_datasets/spatialscaper_RIRs/motusroom_foa.sofa


In [ ]:
import numpy as np
import spatialscaper as ss
import os

NSCAPES = 100  # number of soundscapes to generate
FOREGROUND_DIR = "datasets/sound_event_datasets/FSD50K_FMA"  # directory with FSD50K foreground sound files
RIR_DIR = "datasets/rir_datasets"  # directory containing Room Impulse Response (RIR) files
ROOM = "motusroom"  # initial room setting, change according to available rooms listed below
FORMAT = "foa"  # output format specifier: could be 'mic' or 'foa'
N_EVENTS_MEAN = 15  # mean number of foreground events in a soundscape
N_EVENTS_STD = 6  # standard deviation of the number of foreground events
DURATION = 60.0  # duration in seconds of each soundscape
SR = 24000  # SpatialScaper default sampling rate for the audio files
OUTPUT_DIR = "output"  # Directory to store the generated soundscapes
REF_DB = (-65)  # Reference decibel level for the background ambient noise. Try making this random too!

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, FORMAT), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels"), exist_ok=True)

def generate_soundscape(index):
    print(f"Generating soundscape: {index + 1}/{NSCAPES}") 
    track_name = f"fold1_room1_mix{index+1:03d}"
    ssc = ss.Scaper(
        DURATION,
        FOREGROUND_DIR,
        RIR_DIR,
        FORMAT,
        ROOM,
        max_event_overlap=2,
        speed_limit=2.0,
    )
    ssc.ref_db = REF_DB
    ssc.add_background()

    # Add a random number of foreground events, based on the specified mean and standard deviation.
    n_events = int(np.random.normal(N_EVENTS_MEAN, N_EVENTS_STD))
    n_events = n_events if n_events > 0 else 1
    print(f"Number of events to add: {n_events}")
    for i in range(n_events):
        print(f"Adding event {i + 1}/{n_events}") 
        ssc.add_event()  # randomly choosing and spatializing an FSD50K sound event

    audiofile = os.path.join(OUTPUT_DIR, FORMAT, track_name)
    labelfile = os.path.join(OUTPUT_DIR, "labels", track_name)
    ssc.generate(audiofile, labelfile)

for iscape in range(NSCAPES):
    generate_soundscape(iscape)

Generating soundscape: 1/100
Number of events to add: 15
Adding event 1/15
Adding event 2/15
Adding event 3/15
Adding event 4/15
Adding event 5/15
Adding event 6/15
Adding event 7/15
Adding event 8/15
Adding event 9/15
Adding event 10/15
Adding event 11/15
Adding event 12/15
Adding event 13/15
Adding event 14/15
Adding event 15/15


/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/laughter/train/Laughter/262426.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(
/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/waterTap/train/Water_tap_and_faucet/185769.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(


Generating soundscape: 2/100
Number of events to add: 13
Adding event 1/13
Adding event 2/13
Adding event 3/13
Adding event 4/13
Adding event 5/13
Adding event 6/13
Adding event 7/13
Adding event 8/13
Adding event 9/13
Adding event 10/13
Adding event 11/13
Adding event 12/13
Adding event 13/13
Generating soundscape: 3/100
Number of events to add: 18
Adding event 1/18
Adding event 2/18
Adding event 3/18
Adding event 4/18
Adding event 5/18
Adding event 6/18
Adding event 7/18
Adding event 8/18
Adding event 9/18
Adding event 10/18
Adding event 11/18
Adding event 12/18
Adding event 13/18
Adding event 14/18
Adding event 15/18
Adding event 16/18
Adding event 17/18
Adding event 18/18


/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/music/test/Pop/007526.mp3" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(
/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/music/train/Pop/003535.mp3" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(


Generating soundscape: 4/100
Number of events to add: 16
Adding event 1/16
Adding event 2/16
Adding event 3/16
Adding event 4/16
Adding event 5/16
Adding event 6/16
Adding event 7/16
Adding event 8/16
Adding event 9/16
Adding event 10/16
Adding event 11/16
Adding event 12/16
Adding event 13/16
Adding event 14/16
Adding event 15/16
Adding event 16/16
Generating soundscape: 5/100
Number of events to add: 20
Adding event 1/20
Adding event 2/20
Adding event 3/20
Adding event 4/20
Adding event 5/20
Adding event 6/20
Adding event 7/20
Adding event 8/20
Adding event 9/20
Adding event 10/20
Adding event 11/20
Adding event 12/20
Adding event 13/20
Adding event 14/20
Adding event 15/20
Adding event 16/20
Adding event 17/20
Adding event 18/20
Adding event 19/20
Adding event 20/20


...

/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/waterTap/train/Water_tap_and_faucet/129321.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(


Generating soundscape: 27/100
Number of events to add: 23
Adding event 1/23
Adding event 2/23
Adding event 3/23
Adding event 4/23
Adding event 5/23
Adding event 6/23
Adding event 7/23
Adding event 8/23
Adding event 9/23
Adding event 10/23
Adding event 11/23
Adding event 12/23
Adding event 13/23
Adding event 14/23
Adding event 15/23
Adding event 16/23
Adding event 17/23
Adding event 18/23
Adding event 19/23
Adding event 20/23
Adding event 21/23
Adding event 22/23
Adding event 23/23


/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/maleSpeech/train/Male_speech_and_man_speaking/124834.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(
/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/telephone/train/Telephone_bell_ringing/274289.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(


Generating soundscape: 28/100
Number of events to add: 16
Adding event 1/16
Adding event 2/16
Adding event 3/16
Adding event 4/16
Adding event 5/16
Adding event 6/16
Adding event 7/16
Adding event 8/16
Adding event 9/16
Adding event 10/16
Adding event 11/16
Adding event 12/16
Adding event 13/16
Adding event 14/16
Adding event 15/16
Adding event 16/16
Generating soundscape: 29/100
Number of events to add: 18
Adding event 1/18
Adding event 2/18
Adding event 3/18
Adding event 4/18
Adding event 5/18
Adding event 6/18
Adding event 7/18
Adding event 8/18
Adding event 9/18
Adding event 10/18
Adding event 11/18
Adding event 12/18
Adding event 13/18
Adding event 14/18
Adding event 15/18
Adding event 16/18
Adding event 17/18
Adding event 18/18


/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/doorCupboard/test/Cupboard_open_or_close/124431.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(
/Users/karinna_cv/SpatialScaper/SpatialScaper/spatialscaper/core.py:304: UserWarning: Could not find a start time for sound event "datasets/sound_event_datasets/FSD50K_FMA/footsteps/test/Walk_and_footsteps/137172.wav" that satisfies max_event_overlap = 2. If this continues happening, you may want to consider adding less sound events to the scape or increasing max_event_overlap.
  warnings.warn(
